In [ ]:
# File to investigate hyperparamter configurations

In [ ]:
import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For preprocessing
from sklearn.model_selection import StratifiedShuffleSplit, ParameterGrid
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, FunctionTransformer
from sklearn_pandas import DataFrameMapper

import torch # For building the networks 
import torchtuples as tt # Some useful functions
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [ ]:
df_complete = pyreadr.read_r("/home/jupyter-niclas/Approach_2_Including_HbA1c_t0/Parameter_tuning/input_cox.rds")
# Coverting R-file as panda
df = df_complete[None]
df.head()

In [ ]:
# Using stratification to make sure the event-ratio is constant over subsets
event_col = 'event' 

# First split: 80% train+val, 20% test
sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1234)
for train_val_idx, test_idx in sss1.split(df, df[event_col]):
    df_train_val = df.iloc[train_val_idx].reset_index(drop=True)
    df_test = df.iloc[test_idx].reset_index(drop=True)

# Second split: 20% of train_val becomes validation → 16% of full set
sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, val_idx in sss2.split(df_train_val, df_train_val[event_col]):
    df_train = df_train_val.iloc[train_idx].reset_index(drop=True)
    df_val = df_train_val.iloc[val_idx].reset_index(drop=True)

In [ ]:
# Check event rate
def check_event_rate(df, name):
    rate = df[event_col].mean()
    print(f"{name} event rate: {rate:.3f} ({df[event_col].sum()}/{len(df)})")

check_event_rate(df_train, "Train")
check_event_rate(df_val, "Validation")
check_event_rate(df_test, "Test")

In [ ]:
# Feature transformations
cols_standardize = ['val_HbA1c', 'age', 'BMI', 'systolic_blood_pressure', 'diastolic_blood_pressure', 'cholesterol_LDL', 'triglycerides', 'ACR']
cols_categorical = ['sexe', 'smoking_status', 'alcohol_risk_consumption', 'qmedea', 'cat_eGFR']

# Standardize numeric features
standardize = [([col], StandardScaler()) for col in cols_standardize]

# One-hot encode categorical (multi-class) features
categorical = [([col], OneHotEncoder(sparse_output=False, handle_unknown='ignore')) for col in cols_categorical]

# Combine 
x_mapper = DataFrameMapper(standardize + categorical, df_out=True)

In [ ]:
x_train = x_mapper.fit_transform(df_train).to_numpy().astype('float32')
x_val = x_mapper.transform(df_val).to_numpy().astype('float32')
x_test = x_mapper.transform(df_test).to_numpy().astype('float32')

In [ ]:
# set targets
get_target = lambda df: (df['time_to_event'].values, df['event'].values)
y_train = get_target(df_train)
y_val = get_target(df_val)
durations_test, events_test = get_target(df_test)
val = x_val, y_val

In [ ]:
# Assumes these variables are defined already:
# x_train, y_train, x_val, y_val, x_test, durations_test, events_test

# Define grid
param_grid = {
    'num_nodes': [[32], [64], [64, 32]],
    'dropout': [0.0, 0.1, 0.3],
    'batch_size': [128, 256],
    'learning_rate': [1e-3, 1e-4],
    'batch_norm': [True]
}

In [ ]:
grid = list(ParameterGrid(param_grid))
results = []

for i, params in enumerate(grid):
    print(f"\n Running configuration {i+1}/{len(grid)}: {params}")
    
    # Build model
    net = tt.practical.MLPVanilla(
        in_features=x_train.shape[1],
        num_nodes=params['num_nodes'],
        out_features=1,
        batch_norm=params['batch_norm'],
        dropout=params['dropout'],
        activation=torch.nn.ReLU
    )
 
    model = CoxPH(net, tt.optim.Adam)
    model.optimizer.set_lr(params['learning_rate'])
    
    # Fit model
    callbacks = [tt.callbacks.EarlyStopping(patience=10)]
    log = model.fit(
        x_train, y_train,
        batch_size=params['batch_size'],
        epochs=256,
        callbacks=callbacks,
        verbose=False,
        val_data=(x_val, y_val),
        val_batch_size=params['batch_size']
    )

    model.compute_baseline_hazards()
    
    # Evaluate: Train
    surv_train = model.predict_surv_df(x_train)
    ev_train = EvalSurv(surv_train, y_train[0], y_train[1], censor_surv='km')
    c_train = ev_train.concordance_td()

    # Evaluate: Validation
    surv_val = model.predict_surv_df(x_val)
    ev_val = EvalSurv(surv_val, y_val[0], y_val[1], censor_surv='km')
    c_val = ev_val.concordance_td()

    # Evaluate: Test
    surv_test = model.predict_surv_df(x_test)
    ev_test = EvalSurv(surv_test, durations_test, events_test, censor_surv='km')
    c_test = ev_test.concordance_td()
    
    # Overfitting indicator
    overfit_gap = c_train - c_val

    print(f"C-index | Train: {c_train:.4f} | Val: {c_val:.4f} | Test: {c_test:.4f} | Gap: {overfit_gap:.4f}")


    results.append({
        **params,
        'c_index_train': c_train,
        'c_index_val': c_val,
        'c_index_test': c_test,
        'overfit_gap': overfit_gap
    })

In [ ]:
# Create and show results
results_df = pd.DataFrame(results).sort_values(by='c_index_val', ascending=False)
from IPython.display import display
display(results_df)

In [ ]:
# Save DataFrame as CSV
results_df.to_csv("Deep_Cox_parameter_tuning_copy_#1_results.csv", index=False)
print("Results saved as: Deep_Cox_parameter_tuning_copy_#1_results.csv")

In [ ]:
results_df['config'] = results_df.apply(lambda row: 
    f"LR:{row['learning_rate']},BS:{row['batch_size']},DO:{row['dropout']},NN:{row['num_nodes']}", axis=1
)

# Sort configs by validation C-index (descending)
results_df = results_df.sort_values(by='c_index_val', ascending=False).reset_index(drop=True)

# --- Step 3: Plot C-index scores ---
plt.figure(figsize=(14, 6))

plt.plot(results_df['config'], results_df['c_index_train'], marker='^', label='Train C-index', linestyle='-')
plt.plot(results_df['config'], results_df['c_index_val'], marker='o', label='Validation C-index', linestyle='--')
plt.plot(results_df['config'], results_df['c_index_test'], marker='s', label='Test C-index', linestyle='-.')

plt.ylabel("C-index")
plt.xticks(rotation=45, ha='right')
plt.grid(True, linestyle=':', alpha=0.5)
plt.legend()
plt.tight_layout()

plt.savefig("Deep_Cox_copy_param_search_plot.jpg", dpi=300)
print("Plot saved as: Deep_Cox_copy_param_search_plot.jpg")
plt.show()

In [ ]:
top10_df = results_df.sort_values(by='c_index_val', ascending=False).head(10).reset_index(drop=True)

# Plot
plt.figure(figsize=(14, 6))

plt.plot(top10_df['config'], top10_df['c_index_train'], marker='^', label='Train C-index', linestyle='-')
plt.plot(top10_df['config'], top10_df['c_index_val'], marker='o', label='Validation C-index', linestyle='--')
plt.plot(top10_df['config'], top10_df['c_index_test'], marker='s', label='Test C-index', linestyle='-.')

plt.ylabel("C-index", fontsize=18)
plt.xticks([]) 
plt.grid(True, linestyle=':', alpha=0.5)

# Horizontal legend below the plot
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=3, frameon=False, fontsize=18)
plt.tight_layout()

plt.savefig("Deep_Cox_top10_param_search_plot.jpg", dpi=300, bbox_inches='tight')
print("Plot saved as: Deep_Cox_top10_param_search_plot.jpg")
plt.show()

In [ ]:
# Parameter testing #2: focus on layer-depth
# Define the range of architectures to test
layer_configs = [
    [32],
    [64],
    [128],
    [64, 32],
    [128, 64],
    [128, 64, 32],
    [256, 128, 64]
]

results = []

In [ ]:
# Fixed hyperparams
fixed_dropout = 0.3
fixed_batch_size = 128
fixed_learning_rate = 1e-3
fixed_batch_norm = True

for i, num_nodes in enumerate(layer_configs):
    print(f"\n Running config {i+1}/{len(layer_configs)}: Layers {num_nodes}")
    
    # Model setup
    net = tt.practical.MLPVanilla(
        in_features=x_train.shape[1],
        num_nodes=num_nodes,
        out_features=1,
        batch_norm=fixed_batch_norm,
        dropout=fixed_dropout,
        activation=torch.nn.ReLU
    )
    
    model = CoxPH(net, tt.optim.Adam)
    model.optimizer.set_lr(fixed_learning_rate)
    
    # Train
    callbacks = [tt.callbacks.EarlyStopping(patience=10)]
    log = model.fit(
        x_train, y_train,
        batch_size=fixed_batch_size,
        epochs=256,
        callbacks=callbacks,
        verbose=False,
        val_data=(x_val, y_val),
        val_batch_size=fixed_batch_size
    )
    
    model.compute_baseline_hazards()
    
    # Evaluation
    ev_train = EvalSurv(model.predict_surv_df(x_train), y_train[0], y_train[1], censor_surv='km')
    c_train = ev_train.concordance_td()
    
    ev_val = EvalSurv(model.predict_surv_df(x_val), y_val[0], y_val[1], censor_surv='km')
    c_val = ev_val.concordance_td()
    
    ev_test = EvalSurv(model.predict_surv_df(x_test), durations_test, events_test, censor_surv='km')
    c_test = ev_test.concordance_td()
    
    overfit_gap = c_train - c_val
    
    print(f" C-index | Train: {c_train:.4f} | Val: {c_val:.4f} | Test: {c_test:.4f} | Gap: {overfit_gap:.4f}")

    results.append({
        'num_nodes': str(num_nodes),
        'c_index_train': c_train,
        'c_index_val': c_val,
        'c_index_test': c_test,
        'overfit_gap': overfit_gap
    })

In [ ]:
# Create and show results
results_df = pd.DataFrame(results).sort_values(by='c_index_val', ascending=False)
display(results_df)

In [ ]:
# Save DataFrame as CSV
results_df.to_csv("Deep_Cox_parameter_tuning_copy_#2_layer_depth_results.csv", index=False)
print("Results saved as: Deep_Cox_parameter_tuning_copy_#2_layer_depth_results.csv")

In [ ]:
# Plot C-index (val and test) vs layer configuration
plt.figure(figsize=(10, 6))
x_labels = results_df['num_nodes']

plt.plot(x_labels, results_df['c_index_val'], marker='o', label='Validation C-index')
plt.plot(x_labels, results_df['c_index_test'], marker='s', label='Test C-index')
plt.plot(x_labels, results_df['c_index_train'], marker='^', label='Train C-index')


plt.xlabel("Layer Configuration (num_nodes)")
plt.ylabel("C-index")
plt.title("C-index vs. Network Depth")
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()

plt.savefig("Deep_Cox_layer_depth_test_plot.jpg", dpi=300)
print("Plot saved as: Deep_Cox_layer_depth_test_plot.jpg")
plt.show()

In [ ]:
# Parameter testing #3
# Define architectural parameter grid
param_grid = {
    'layer_config': [[32], [64], [64, 32], [64, 64], [128, 64]],
    'activation': [torch.nn.ReLU, torch.nn.ELU, torch.nn.Tanh],
    'batch_norm': [True, False]
}
grid = list(ParameterGrid(param_grid))
results = []

# Fixed parameters from previous tuning
learning_rate = 1e-3
batch_size = 128
dropout = 0.3

for i, params in enumerate(grid):
    print(f"\n Running architecture configuration {i+1}/{len(grid)}: {params}")

    # Build model
    net = tt.practical.MLPVanilla(
        in_features=x_train.shape[1],
        num_nodes=params['layer_config'],
        out_features=1,
        batch_norm=params['batch_norm'],
        dropout=dropout,
        activation=params['activation']
    )

    model = CoxPH(net, tt.optim.Adam)
    model.optimizer.set_lr(learning_rate)

    # Fit model
    callbacks = [tt.callbacks.EarlyStopping(patience=10)]
    log = model.fit(
        x_train, y_train,
        batch_size=batch_size,
        epochs=256,
        callbacks=callbacks,
        verbose=False,
        val_data=(x_val, y_val),
        val_batch_size=batch_size
    )

    model.compute_baseline_hazards()

    # Evaluate: Train
    surv_train = model.predict_surv_df(x_train)
    ev_train = EvalSurv(surv_train, y_train[0], y_train[1], censor_surv='km')
    c_train = ev_train.concordance_td()

    # Evaluate: Validation
    surv_val = model.predict_surv_df(x_val)
    ev_val = EvalSurv(surv_val, y_val[0], y_val[1], censor_surv='km')
    c_val = ev_val.concordance_td()

    # Evaluate: Test
    surv_test = model.predict_surv_df(x_test)
    ev_test = EvalSurv(surv_test, durations_test, events_test, censor_surv='km')
    c_test = ev_test.concordance_td()

    overfit_gap = c_train - c_val

    print(f"C-index | Train: {c_train:.4f} | Val: {c_val:.4f} | Test: {c_test:.4f} | Gap: {overfit_gap:.4f}")

    results.append({
        'layer_config': params['layer_config'],
        'activation': params['activation'].__name__,
        'batch_norm': params['batch_norm'],
        'c_index_train': c_train,
        'c_index_val': c_val,
        'c_index_test': c_test,
        'overfit_gap': overfit_gap
    })

In [ ]:
# Create DataFrame and sort
results_df = pd.DataFrame(results)
results_df['config'] = results_df.apply(lambda row: 
    f"L:{row['layer_config']},A:{row['activation']},BN:{row['batch_norm']}", axis=1)
results_df = results_df.sort_values(by='c_index_val', ascending=False).reset_index(drop=True)

# Display and save
from IPython.display import display
display(results_df)
results_df.to_csv("Deep_Cox_architecture_analysis_results.csv", index=False)
print("Results saved as: Deep_Cox_architecture_analysis_results.csv")

# Plot C-index scores for top 10 configs
top10_df = results_df.sort_values(by='c_index_val', ascending=False).head(10).reset_index(drop=True)
plt.figure(figsize=(14, 6))

plt.plot(top10_df['config'], top10_df['c_index_train'], marker='^', label='Train C-index', linestyle='-')
plt.plot(top10_df['config'], top10_df['c_index_val'], marker='o', label='Validation C-index', linestyle='--')
plt.plot(top10_df['config'], top10_df['c_index_test'], marker='s', label='Test C-index', linestyle='-.')

plt.ylabel("C-index", fontsize=18)
plt.xticks([]) 
plt.grid(True, linestyle=':', alpha=0.5)

# Horizontal legend below the plot
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=3, frameon=False, fontsize=18)
plt.tight_layout()

plt.savefig("Deep_Cox_top10_architecture_analysis_plot.jpg", dpi=300)
print("Plot saved as: Deep_Cox_architecture_analysis_plot.jpg")
plt.show()